<a href="https://colab.research.google.com/github/nickprock/falkon/blob/master/notebooks/falkon_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Falkon on Colab

The goal of this notebook is to show how to install Falkon on Colab. Below is also an example of classification on the moons dataset.

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
!pip install pykeops[colab] > install.log

In [ ]:
!git clone --recurse-submodules https://github.com/FalkonML/falkon.git

In [ ]:
!cd falkon/
!pip install falkon/keops/

In [ ]:
!pip install falkon/

## Binary Classification

In this example, two kernels were compared to classify the moons dataset. For more information about model hyperparameters read the [official documentation](https://falkonml.github.io/falkon/index.html).

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import falkon

### Generate the dataset

In [ ]:
X, Y = make_moons(n_samples=100000, noise = 0.1,random_state=42)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(X[Y == 0,0], X[Y == 0,1], alpha=0.1, marker='.')
_ = ax.scatter(X[Y == 1,0], X[Y == 1,1], alpha=0.1, marker='.')

### Train-Test split and Data Preprocessing

For more informations on data preprocessing read the [docs](https://falkonml.github.io/falkon/examples/logistic_falkon.html#Data-Preprocessing).

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42, shuffle=True)

In [ ]:
X_train = torch.from_numpy(X_train).to(dtype=torch.float32)
X_test = torch.from_numpy(X_test).to(dtype=torch.float32)
Y_train = torch.from_numpy(Y_train).to(dtype=torch.float32).reshape(-1, 1)
Y_test = torch.from_numpy(Y_test).to(dtype=torch.float32).reshape(-1, 1)

In [ ]:
Y_train[Y_train == 0] = -1
Y_test[Y_test == 0] = -1

### Define the Falkon Model

We compare two kernels with the same hyperparameters.

In [ ]:
def binary_loss(true, pred):
    return torch.mean((true != torch.sign(pred)).to(torch.float32))

In [ ]:
options = falkon.FalkonOptions(use_cpu=True)

In [ ]:
flk_Gaussian = falkon.Falkon(kernel=falkon.kernels.GaussianKernel( sigma=5), penalty=1e-5, M=1000, options=options)
flk_Gaussian.fit(X_train, Y_train)

In [ ]:
flk_Laplacian = falkon.Falkon(kernel=falkon.kernels.LaplacianKernel( sigma=5), penalty=1e-5, M=1000, options=options)
flk_Laplacian.fit(X_train, Y_train)

### Test

In [ ]:
predsG = flk_Gaussian.predict(X_test)
test_err = binary_loss(Y_test, predsG)
print("Error: %.2f%%" % (test_err*100))

In [ ]:
predsL = flk_Laplacian.predict(X_test)
test_err = binary_loss(Y_test, predsL)
print("Error: %.2f%%" % (test_err*100))

### Plot the results

In [ ]:
def plot_predictions(preds, ax):
    ax.scatter(X_test[((Y_test == -1) & (preds.sign() == Y_test)).reshape(-1), 0],
               X_test[((Y_test == -1) & (preds.sign() == Y_test)).reshape(-1), 1],
               alpha=0.1, marker='.', color='b')
    ax.scatter(X_test[((Y_test == 1) & (preds.sign() == Y_test)).reshape(-1),0],
               X_test[((Y_test == 1) & (preds.sign() == Y_test)).reshape(-1),1],
               alpha=0.1, marker='.', color='r')

    ax.scatter(X_test[((Y_test == -1) & (preds.sign() != Y_test)).reshape(-1), 0],
               X_test[((Y_test == -1) & (preds.sign() != Y_test)).reshape(-1), 1],
               alpha=0.2, marker='x', color='green')
    ax.scatter(X_test[((Y_test == 1) & (preds.sign() != Y_test)).reshape(-1),0],
               X_test[((Y_test == 1) & (preds.sign() != Y_test)).reshape(-1),1],
               alpha=0.2, marker='x', color='gold')


In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(13, 5))
plot_predictions(predsG, ax[0])
ax[0].set_title("Gaussian")
plot_predictions(predsL, ax[1])
_ = ax[1].set_title("Laplacian")